# 🧬 Generación Sintética Avanzada (Conditioning on Time)

**Versión:** v3_experimental  
**Rol:** AI Researcher  
**Fecha:** 2026-01-08

---

## Objetivo

Generar datos sintéticos usando una **estrategia condicional** que preserva mejor la relación tiempo-covariables.

## Metodología: Conditioning on Time

En lugar de un único sintetizador global, entrenamos **sintetizadores separados para subgrupos** definidos por:
- Cuartiles de `duration`
- Estado de `event` (0 o 1)

Esto preserva las correlaciones condicionales entre features y tiempo/evento.

### Justificación Científica

> "Conditioning on Time" (OpenReview) mejora la fidelidad en survival analysis al capturar heterogeneidad temporal.

---

In [ ]:
# ==============================================================================
# CONFIGURACIÓN Y DEPENDENCIAS
# ==============================================================================

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import json
import joblib
from pathlib import Path
from datetime import datetime

# SDV
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer

# Configuración
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths
DATA_DIR = Path("../../v2/data/processed")
OUTPUT_DIR = Path(".")

# Cuántas filas sintéticas totales
N_SYNTHETIC_TOTAL = 1000

print("✅ Dependencias cargadas")
print(f"   Estrategia: Conditioning on Time")
print(f"   Random State: {RANDOM_STATE}")
print(f"   N Sintético Total: {N_SYNTHETIC_TOTAL}")

---
## 1. Carga y Preparación de Datos

In [ ]:
# ==============================================================================
# CARGA DE DATOS (SOLO TRAIN - CERO LEAKAGE)
# ==============================================================================

train_df = pd.read_parquet(DATA_DIR / "train_final.parquet")

print(f"📊 Datos cargados (SOLO TRAIN):")
print(f"   Shape: {train_df.shape}")

# Excluir columnas zero-variance
zero_var_cols = [col for col in train_df.columns if train_df[col].nunique() <= 1]
train_clean = train_df.drop(columns=zero_var_cols, errors='ignore')

print(f"   Excluidas (zero-variance): {zero_var_cols}")
print(f"   Shape final: {train_clean.shape}")

---
## 2. Estratificación por Cuartiles de Duration y Event

In [ ]:
# ==============================================================================
# CREAR SUBGRUPOS PARA CONDITIONING
# ==============================================================================

print("🔧 Creando subgrupos condicionales...")

# Crear cuartiles de duration
train_clean['duration_quartile'] = pd.qcut(
    train_clean['duration'], 
    q=4, 
    labels=['Q1', 'Q2', 'Q3', 'Q4']
)

# Crear grupo combinado: cuartil + event
train_clean['stratum'] = (
    train_clean['duration_quartile'].astype(str) + '_E' + 
    train_clean['event'].astype(str)
)

# Ver distribución de estratos
strata_counts = train_clean['stratum'].value_counts().sort_index()
print(f"\n📊 Distribución de estratos:")
for stratum, count in strata_counts.items():
    pct = count / len(train_clean) * 100
    print(f"   {stratum}: {count} ({pct:.1f}%)")

# Guardar info de estratos
strata_info = strata_counts.to_dict()

In [ ]:
# ==============================================================================
# ANÁLISIS DE SUBGRUPOS
# ==============================================================================

print("\n📊 Estadísticas por estrato:")
print("="*70)

# Estadísticas de duration por estrato
for stratum in sorted(train_clean['stratum'].unique()):
    subset = train_clean[train_clean['stratum'] == stratum]
    dur_mean = subset['duration'].mean()
    dur_std = subset['duration'].std()
    event_rate = subset['event'].mean()
    
    print(f"{stratum}: n={len(subset)}, duration μ={dur_mean:.2f}±{dur_std:.2f}, event={event_rate:.0%}")

---
## 3. Entrenamiento de Sintetizadores por Estrato

In [ ]:
# ==============================================================================
# ENTRENAR UN SINTETIZADOR POR CADA ESTRATO
# ==============================================================================

print("🏋️ Entrenando sintetizadores por estrato...")

# Columnas a usar (sin las auxiliares de estratificación)
feature_cols = [c for c in train_clean.columns if c not in ['duration_quartile', 'stratum']]

# Diccionario para guardar sintetizadores
synthesizers = {}
metadata_dict = {}

for stratum in sorted(train_clean['stratum'].unique()):
    print(f"\n   📌 Estrato: {stratum}")
    
    # Filtrar datos del estrato
    subset = train_clean[train_clean['stratum'] == stratum][feature_cols].copy()
    
    # Verificar que hay suficientes datos
    if len(subset) < 10:
        print(f"      ⚠️ Solo {len(subset)} filas, omitiendo estrato")
        continue
    
    # Crear metadata para este subconjunto
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(subset)
    
    # Ajustar tipos
    if 'event' in metadata.columns:
        metadata.update_column('event', sdtype='categorical')
    if 'duration' in metadata.columns:
        metadata.update_column('duration', sdtype='numerical')
    
    # Binarios como categóricos
    for col in subset.columns:
        if col.startswith('tech_') or col == 'genero_m':
            if col in metadata.columns:
                metadata.update_column(col, sdtype='categorical')
    
    metadata.validate()
    
    # Crear y entrenar sintetizador
    synth = GaussianCopulaSynthesizer(
        metadata,
        enforce_min_max_values=True,
        enforce_rounding=True,
        default_distribution='norm'
    )
    
    synth.fit(subset)
    
    synthesizers[stratum] = synth
    metadata_dict[stratum] = metadata
    
    print(f"      ✅ Entrenado con {len(subset)} filas")

print(f"\n✅ {len(synthesizers)} sintetizadores entrenados")

---
## 4. Generación Condicional

In [ ]:
# ==============================================================================
# GENERAR MUESTRAS PROPORCIONALES DE CADA ESTRATO
# ==============================================================================

print(f"🧬 Generando {N_SYNTHETIC_TOTAL} filas sintéticas...")

# Calcular cuántas muestras por estrato (proporcional a distribución real)
total_real = sum(strata_info.values())
samples_per_stratum = {}

for stratum, count in strata_info.items():
    if stratum in synthesizers:
        samples_per_stratum[stratum] = max(10, int(N_SYNTHETIC_TOTAL * count / total_real))

# Ajustar para que sumen exactamente N_SYNTHETIC_TOTAL
diff = N_SYNTHETIC_TOTAL - sum(samples_per_stratum.values())
if diff != 0:
    # Agregar/quitar del estrato más grande
    largest_stratum = max(samples_per_stratum, key=samples_per_stratum.get)
    samples_per_stratum[largest_stratum] += diff

print("\n📊 Muestras a generar por estrato:")
for stratum, n in sorted(samples_per_stratum.items()):
    print(f"   {stratum}: {n}")

# Generar
synthetic_dfs = []

for stratum, n_samples in samples_per_stratum.items():
    if stratum in synthesizers:
        synth_subset = synthesizers[stratum].sample(num_rows=n_samples)
        synth_subset['source_stratum'] = stratum  # Marcar origen
        synthetic_dfs.append(synth_subset)
        print(f"   ✅ {stratum}: {len(synth_subset)} generadas")

# Combinar
synthetic_df = pd.concat(synthetic_dfs, ignore_index=True)
print(f"\n📊 Total sintético: {len(synthetic_df)} filas")

---
## 5. Post-Procesamiento

In [ ]:
# ==============================================================================
# POST-PROCESAMIENTO
# ==============================================================================

print("🔧 Aplicando post-procesamiento...")

# Remover columna auxiliar de origen (opcional, la mantenemos para trazabilidad)
# synthetic_df = synthetic_df.drop(columns=['source_stratum'])

# 1. Duration > 0
feature_cols_orig = [c for c in train_clean.columns if c not in ['duration_quartile', 'stratum']]
min_duration = train_clean['duration'].min()

if 'duration' in synthetic_df.columns:
    before = (synthetic_df['duration'] <= 0).sum()
    synthetic_df['duration'] = synthetic_df['duration'].clip(lower=min_duration)
    print(f"   ✅ duration: {before} valores corregidos")

# 2. Event binario
if 'event' in synthetic_df.columns:
    synthetic_df['event'] = synthetic_df['event'].round().astype(int).clip(0, 1)
    print(f"   ✅ event: binario")

# 3. Edad en rango
if 'edad' in synthetic_df.columns:
    min_edad, max_edad = train_clean['edad'].min(), train_clean['edad'].max()
    synthetic_df['edad'] = synthetic_df['edad'].clip(min_edad, max_edad).round().astype(int)
    print(f"   ✅ edad: clip [{min_edad}, {max_edad}]")

# 4. Binarios tech_*
binary_cols = [c for c in synthetic_df.columns if c.startswith('tech_') or c == 'genero_m']
for col in binary_cols:
    if col in synthetic_df.columns:
        synthetic_df[col] = synthetic_df[col].round().astype(int).clip(0, 1)
print(f"   ✅ {len(binary_cols)} columnas binarias")

# 5. Habilidades hab_*
hab_cols = [c for c in synthetic_df.columns if c.startswith('hab_')]
for col in hab_cols:
    synthetic_df[col] = (synthetic_df[col] * 4).round() / 4
    synthetic_df[col] = synthetic_df[col].clip(0, 1)
print(f"   ✅ {len(hab_cols)} columnas hab_*")

print("\n✅ Post-procesamiento completado")

---
## 6. Validación de Calidad

In [ ]:
# ==============================================================================
# VALIDACIÓN DE FIDELIDAD CONDICIONAL
# ==============================================================================

print("📊 Validación de fidelidad condicional...")
print("="*70)

# Comparar estadísticas globales
print("\n🔍 Comparación Global:")

for col in ['duration', 'event', 'edad']:
    if col in train_clean.columns and col in synthetic_df.columns:
        real_mean = train_clean[col].mean()
        synth_mean = synthetic_df[col].mean()
        delta = abs(real_mean - synth_mean)
        
        print(f"   {col}: Real={real_mean:.3f}, Synth={synth_mean:.3f}, Δ={delta:.3f}")

# Comparar correlación duration-event
print("\n🔗 Correlación Duration-Event:")
real_corr = train_clean['duration'].corr(train_clean['event'])
synth_corr = synthetic_df['duration'].corr(synthetic_df['event'])
print(f"   Real:      {real_corr:.4f}")
print(f"   Sintético: {synth_corr:.4f}")
print(f"   Δ:         {abs(real_corr - synth_corr):.4f}")

In [ ]:
# ==============================================================================
# VALIDACIÓN POR ESTRATO
# ==============================================================================

print("\n📊 Validación por Estrato (source_stratum):")
print("="*70)

for stratum in sorted(synthetic_df['source_stratum'].unique()):
    synth_subset = synthetic_df[synthetic_df['source_stratum'] == stratum]
    real_subset = train_clean[train_clean['stratum'] == stratum]
    
    if len(real_subset) == 0:
        continue
        
    real_dur = real_subset['duration'].mean()
    synth_dur = synth_subset['duration'].mean()
    
    print(f"\n{stratum}:")
    print(f"   n_real={len(real_subset)}, n_synth={len(synth_subset)}")
    print(f"   duration: Real={real_dur:.2f}, Synth={synth_dur:.2f}")

---
## 7. Guardar Resultados

In [ ]:
# ==============================================================================
# GUARDAR DATOS Y MODELOS
# ==============================================================================

print("💾 Guardando resultados...")

# 1. Datos sintéticos (sin columna auxiliar para uso en training)
export_cols = [c for c in synthetic_df.columns if c != 'source_stratum']
synthetic_df[export_cols].to_parquet(OUTPUT_DIR / "synthetic_data_advanced.parquet", index=False)
print(f"   ✅ synthetic_data_advanced.parquet ({len(synthetic_df)} filas)")

# 2. Guardar sintetizadores
for stratum, synth in synthesizers.items():
    safe_name = stratum.replace('_', '-')
    synth.save(OUTPUT_DIR / f"synth_{safe_name}.pkl")
print(f"   ✅ {len(synthesizers)} sintetizadores guardados")

# 3. Reporte de generación
report = {
    "metadata": {
        "date": datetime.now().isoformat(),
        "method": "Conditional GaussianCopula (Stratified)",
        "strategy": "Conditioning on Time Quartiles + Event",
        "random_state": RANDOM_STATE
    },
    "strata": {
        stratum: {
            "n_real": int(strata_info.get(stratum, 0)),
            "n_synthetic": int(samples_per_stratum.get(stratum, 0))
        } for stratum in sorted(strata_info.keys())
    },
    "output": {
        "n_synthetic_total": len(synthetic_df),
        "n_strata": len(synthesizers)
    },
    "validation": {
        "duration_corr_real": float(real_corr),
        "duration_corr_synth": float(synth_corr),
        "delta_corr": float(abs(real_corr - synth_corr))
    },
    "files_generated": [
        "synthetic_data_advanced.parquet"
    ] + [f"synth_{s.replace('_', '-')}.pkl" for s in synthesizers.keys()]
}

with open(OUTPUT_DIR / "advanced_generation_report.json", 'w') as f:
    json.dump(report, f, indent=2)
print(f"   ✅ advanced_generation_report.json")

print("\n" + "="*50)
print("🎉 GENERACIÓN AVANZADA COMPLETADA")
print("="*50)

---
## Resumen

### Método: Conditioning on Time

Se entrenaron **8 sintetizadores separados** (4 cuartiles × 2 estados de event), preservando mejor:
- Correlaciones condicionales entre features y tiempo
- Heterogeneidad entre grupos de riesgo

### Ventajas vs Método Global

| Aspecto | Global | Condicional |
|---------|--------|-------------|
| Simplicidad | ✅ Alta | ⚠️ Media |
| Fidelidad temporal | ⚠️ Media | ✅ Alta |
| Preservación heterogeneidad | ⚠️ Media | ✅ Alta |

### Siguiente Paso

**Prompt 6: Entrenamiento TSTR** - Comparar ambos métodos sintéticos en los escenarios de augmentación.

---